In [2]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
import pandas as pd
import xgboost as xgb
import numpy as np
import random, os, datetime, pickle
import scipy
import matplotlib.pyplot as plt
import numpy as np
import csv, math


# tube.csv에 수술 시점 weight, height 추가
df0 = pd.read_csv('tube.csv')
df0.drop(columns=['weight', 'height'], inplace=True)
df = pd.read_csv('demography_revised.csv')
df = df0.merge(df[['opid', 'weight', 'height']], how='left', on='opid', suffixes=('_o',''))

df.loc[df['weight'] <= 1, 'weight'] = None
df.loc[df['weight'] > 200, 'weight'] = None
df.loc[df['height'] <= 30, 'height'] = None
df.loc[df['height'] > 230, 'height'] = None
df['age'] = df['age'].astype(int)
df = df.loc[df['age'] < 19]
print(f'초기 전체 opid: {len(df)}')
df = df.loc[df['airway_tube_type'] == 'plain']
# [nan 'plain' 'RAE(oral)' 'reinforced' 'LMA' 'T-tube' 'CobraPLA', 'double lumen tube' 'RAE(nasal)' 'laser' 'combitube' 'univent']

print(f'airway tube type이 plain이 아닌 경우 제외: {len(df)}')
# age, sex, airway tube size 값이 없는 경우는 제외

df.dropna(subset=['age', 'airway_tube_size'], inplace=True)  # sex는 모든 데이터 다 있음
df['sex'] = (df['sex'] == 'M')
print(f'age, airway_tube_size 결측치 제외: {len(df)}')

# 나이 계산 -> age_cal 열에 추가
df_b = pd.read_csv('birth_sex.csv')
df_b.rename(columns={'생년월일':'birth_date'}, inplace=True)
df_b['birth_date'] = df_b['birth_date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

df_o = pd.read_csv('opdates.csv')
df_o['opdate'] = df_o['opdate'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

df1 = pd.merge(df_o, df_b, how='inner', on='hid')
df1['age_cal'] = (df1['opdate'] - df1['birth_date'])/pd.Timedelta(days=365.2425)

df = pd.merge(df, df1[['opid', 'age_cal', 'opdate', 'birth_date']], how='inner', on='opid')
#df3 = pd.merge(df, df1[['opid', 'age_cal', 'opdate', 'birth_date']], how='left', on='opid')

# inclusion criteria : 소아 10세 미만
#df['age_cal'] = df['age_cal'] + 0.01 # 생일이랑 수술날 같은 경우
df = df.loc[df['age_cal'] < 10-0.01]  # 생일이랑 수술날 같은 경우
df = df.loc[df['age_cal'] > 0]
print(f'10세 이상 제외 : {len(df)}')

# cuffed 여부와 fixed depth 추가
df_t = pd.read_csv('tube_type.csv')
df_t['cuffed'] = (df_t['cuffed'] == 1)

df_f = pd.read_csv('tube_fixed.csv')

# merge 하면서 cuffed 데이터가 없는 경우는 제외
df = df.merge(df_f, how='left', on='opid')
df = df.merge(df_t[['opid', 'cuffed']], how='inner', on='opid')
print(f'cuffed data가 없는 경우는 제외: {len(df)}')


# 중복되는 hid 경우 제외 (첫번째 수술기록만 가져오기)
df = df.merge(df_o[['opid','hid']], how='inner', on='opid')
df = df.loc[df[['hid', 'opid']].groupby('hid')['opid'].idxmin()]
print(f'중복되는 hid는 첫번째 수술 외 제외: {len(df)}')


# age-based formula에 따른 ETT size
OLD_VAR = 'old_tube_size'
# df[OLD_VAR] = np.round((df['age'] / 4 + 4) * 2) / 2
df[OLD_VAR] = df['age'].apply(lambda x: np.round((x / 4 + 4) * 2) / 2 if x >= 2 else (3.5 if x < 1 else 4)) 
df[OLD_VAR] = df.apply(lambda x: x[OLD_VAR] - 0.5 if x['cuffed'] else x[OLD_VAR], axis=1)


# fixed depth를 output
INPUT_VARS = ['age_cal','sex','weight','height', 'cuffed', 'airway_tube_size']
TARGET_VAR = 'fixed'


random.seed(98)
#seed_everything(SEED)
df.dropna(subset=['fixed'], inplace=True)
print(f'depth가 없는 경우 제외: {len(df)}')
df = df.loc[df['fixed']>=8]
print(f'depth가 8cm 미만인 경우 제외: {len(df)}')
df = shuffle(df)
df.reset_index(drop=True, inplace=True)
y = df[[TARGET_VAR]].values.flatten().astype(float)
x = df.loc[:, INPUT_VARS].values.astype(float)
c = df['opid'].values.flatten().astype(int)

# 저장하기
pickle.dump(df, open(f'dataset/ETT_depth_8','wb'))
np.savez(f'dataset/ETT_depth_8.npz', x=x, y=y, c=c)


# training set의 뒤쪽 20%를 test set 으로 사용
nsamp = len(y)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest
x_test = x[-ntest:, :]
y_test = y[-ntest:]
#y_test_old = y_old[-ntest:]
x_train = x[:ntrain, :]
y_train = y[:ntrain]

print(f'x_train: {(x_train).shape}, x_test: {x_test.shape}')

초기 전체 opid: 155719
airway tube type이 plain이 아닌 경우 제외: 78848
age, airway_tube_size 결측치 제외: 78502
10세 이상 제외 : 59352
cuffed data가 없는 경우는 제외: 45067
중복되는 hid는 첫번째 수술 외 제외: 34042
depth가 없는 경우 제외: 30515
depth가 8cm 미만인 경우 제외: 30223
x_train: (24179, 6), x_test: (6044, 6)


In [50]:
dat = np.load(f'dataset/ETT_depth_8.npz')
x, y = dat['x'], dat['y']
x = x[:,0:5]


nsamp = len(y)
ntest = int(nsamp * 0.2)
ntrain = nsamp - ntest
x_test = x[-ntest:, :]
y_test = y[-ntest:]
x_train = x[:ntrain, :]
y_train = y[:ntrain]

print(f'x_train: {(x_train).shape}, x_test: {x_test.shape}')

x_train: (24179, 5), x_test: (6044, 5)


In [51]:
x_train = x_train[:,0:4]
x_test = x_test[:,0:4]

print(f'x_train: {(x_train).shape}, x_test: {x_test.shape}')

x_train: (24179, 4), x_test: (6044, 4)


# PALS guideline

In [3]:
OLD_VAR = 'old_depth1'
df[OLD_VAR] = 3 * df['old_tube_size']
#df[OLD_VAR] = df['airway_tube_size'].apply(lambda x: np.round((x / 4 + 4) * 2) / 2 if x >= 2 else (3.5 if x < 1 else 4)) 
#df[OLD_VAR] = df.apply(lambda x: x[OLD_VAR] - 0.5 if x['cuffed'] else x[OLD_VAR], axis=1)
y_old = df[[OLD_VAR]].values.flatten().astype(float)
y_test_old = y_old[-ntest:]

print('--------------')
print('old model = 3 * age-based ETT size')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_test_old):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_test_old):.3f}')
print(f'mean_absolute_errors: {mean_absolute_error(y_test, y_test_old):.3f}')
print(f'r2_score: {r2_score(y_test, y_test_old):.3f}')

--------------
old model = 3 * age-based ETT size
--------------
explained_variance_score: 0.605
mean_squared_errors: 3.043
mean_absolute_errors: 1.283
r2_score: 0.588


In [4]:
OLD_VAR = 'old_depth1'
# OLD_VAR = 'old_tube_size'
df[OLD_VAR] = 3 * df['airway_tube_size']
#df[OLD_VAR] = df['airway_tube_size'].apply(lambda x: np.round((x / 4 + 4) * 2) / 2 if x >= 2 else (3.5 if x < 1 else 4)) 
#df[OLD_VAR] = df.apply(lambda x: x[OLD_VAR] - 0.5 if x['cuffed'] else x[OLD_VAR], axis=1)
y_old = df[[OLD_VAR]].values.flatten().astype(float)
y_test_old = y_old[-ntest:]

print('--------------')
print('old model = 3 * ETT size')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_test_old):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_test_old):.3f}')
print(f'mean_absolute_errors: {mean_absolute_error(y_test, y_test_old):.3f}')
print(f'r2_score: {r2_score(y_test, y_test_old):.3f}')

--------------
old model = 3 * ETT size
--------------
explained_variance_score: 0.617
mean_squared_errors: 3.048
mean_absolute_errors: 1.331
r2_score: 0.587


In [5]:
OLD_VAR = 'old_depth1'
# OLD_VAR = 'old_tube_size'
df[OLD_VAR] = df['age'] / 2 + 12
#df[OLD_VAR] = df['airway_tube_size'].apply(lambda x: np.round((x / 4 + 4) * 2) / 2 if x >= 2 else (3.5 if x < 1 else 4)) 
#df[OLD_VAR] = df.apply(lambda x: x[OLD_VAR] - 0.5 if x['cuffed'] else x[OLD_VAR], axis=1)
y_old = df[[OLD_VAR]].values.flatten().astype(float)
y_test_old = y_old[-ntest:]

print('--------------')
print('old model = age / 2 + 12')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_test_old):.3f}')
print(f'mean_squared_errors: {mean_squared_error(y_test, y_test_old):.3f}')
print(f'mean_absolute_errors: {mean_absolute_error(y_test, y_test_old):.3f}')
print(f'r2_score: {r2_score(y_test, y_test_old):.3f}')

--------------
old model = age / 2 + 12
--------------
explained_variance_score: 0.581
mean_squared_errors: 3.133
mean_absolute_errors: 1.361
r2_score: 0.575


# XGBR

In [34]:
# 길이 제한 없는 경우
from sklearn.metrics import mean_absolute_error

# age (일단위)
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'max_depth': [3, 4, 5, 7],#[3,4,5],
                'n_estimators': [25, 50, 100, 200, 300],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'subsample': [0.5, 0.8, 1], #[0.5, 0.8, 1],
                'colsample_bytree': [0.5, 0.8], #[0.8, 1],
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 10
gs = GridSearchCV(estimator=xgb.sklearn.XGBRegressor(),
                    n_jobs=-1,
                    verbose=3,
                    param_grid=param_dict, cv=nfold)
gs.fit(x_train, y_train)
model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mse:.3f}')
print(f'mean_absolute_errors: {mae:.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc3 = np.mean((y_pred >= y_test-1) & (y_pred <= y_test+1))
print(f'acc(+-1cm): {acc3:.3f}')

Fitting 10 folds for each of 120 candidates, totalling 1200 fits

========= found hyperparameter =========
{'colsample_bytree': 0.8, 'max_depth': 4, 'n_estimators': 25, 'subsample': 1}
0.8223467297039477
--------------
new model
--------------
explained_variance_score: 0.809
mean_squared_errors: 1.408
mean_absolute_errors: 0.845
r2_score: 0.809
acc(+-1cm): 0.692


In [35]:
# save model
odir_f = f'mae-{mae:.3f}_mse-{mse:.3f}_XGBR-4inputs_{nfold}fold'
odir = f'result/depth_8cm/{odir_f}'
if not os.path.exists(odir):
    os.mkdir(odir)
model.save_model(f'{odir}/model.model')
pickle.dump(gs, open(f'{odir}/gridSearch','wb'))

# 모델에 대한 정보 txt로 저장
pickle.dump(param_dict, open(f'{odir}/param_dict', 'wb'))
f = open(f'{odir}/result.txt', 'w')
f.write(f'classification model')
f.write(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
f.write(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
f.write(f'mean_absolute_errors: {mean_absolute_error(y_test, y_pred):.3f}')
f.write(f'r2_score: {r2_score(y_test, y_pred):.3f}')
f.close()

# RF model

In [46]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer().fit(x_train)
x_train_imputed = imp.transform(x_train)
x_test_imputed = imp.transform(x_test)

print(f'x_train shape: {x_train_imputed.shape}, x_test shape: {x_test_imputed.shape}')

x_train shape: (24179, 5), x_test shape: (6044, 5)


In [47]:
from sklearn.ensemble import RandomForestRegressor

SEED = 98
param_dict = {
                #'learning_rate': [ 0.05, 0.07, 0.1], #[0.01, 0.03, 0.05],
                'n_estimators': [50, 100, 200, 300],
                'max_features': ['auto', 'sqrt', 'log2'],
                #'n_estimators': [100],#[25, 50, 75, 100],
                #'n_estimators': [50],
                'min_samples_split': [2,3,5],
                'min_samples_leaf': [1,2,3],
               'bootstrap': [True, False]
                #'gamma': [0.9], #[0.3, 0.5, 0.7, 0.9],
                #'scale_pos_weight': [5, 10], #[1,10,30,100]
            }
nfold = 10
gs = GridSearchCV(estimator=RandomForestRegressor(random_state = SEED),
                    n_jobs=-1,
                    verbose=3,
                    param_grid=param_dict, cv=nfold)
gs.fit(x_train_imputed, y_train)
#model = gs.best_estimator_.get_booster()

print()
print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")

y_pred = gs.predict(x_test_imputed).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mse:.3f}')
print(f'mean_absolute_errors: {mae:.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc3 = np.mean((y_pred >= y_test-1) & (y_pred <= y_test+1))
print(f'acc(+-1cm): {acc3:.3f}')

Fitting 10 folds for each of 216 candidates, totalling 2160 fits
[CV 6/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.794 total time=   8.4s
[CV 10/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=200;, score=0.818 total time=  16.4s
[CV 10/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200;, score=0.821 total time=  15.5s
[CV 1/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=3, n_estimators=50;, score=0.799 total time=   3.8s
[CV 1/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=3, n_estimators=100;, score=0.800 total time=   7.5s
[CV 1/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=3, n_estimators=300;, score=0.801 total time=  22.5s
[CV 1/10] END bootstrap=True, max_features=auto, min_samples_leaf=3, min_samples_split=2, n_es

In [40]:
# 4 input model (age, sex, weight, height)
y_pred = gs.predict(x_test_imputed).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("========= found hyperparameter =========")
print(gs.best_params_)
print(gs.best_score_)
print("========================================")
print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mse:.3f}')
print(f'mean_absolute_errors: {mae:.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc3 = np.mean((y_pred >= y_test-1) & (y_pred <= y_test+1))
print(f'acc(+-1cm): {acc3:.3f}')

========= found hyperparameter =========
{'bootstrap': True, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 300}
0.8139923996417548
--------------
new model
--------------
explained_variance_score: 0.800
mean_squared_errors: 1.474
mean_absolute_errors: 0.871
r2_score: 0.800
acc(+-1cm): 0.674


In [48]:
# 5 inputs model
rf = RandomForestRegressor(bootstrap= True, max_features= 'sqrt', min_samples_leaf= 3, min_samples_split = 2, n_estimators= 300)
rf.fit(x_train_imputed, y_train)

y_pred = gs.predict(x_test_imputed).flatten()
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('--------------')
print('new model')
print('--------------')
print(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
print(f'mean_squared_errors: {mse:.3f}')
print(f'mean_absolute_errors: {mae:.3f}')
print(f'r2_score: {r2_score(y_test, y_pred):.3f}')
# accuracy
acc3 = np.mean((y_pred >= y_test-1) & (y_pred <= y_test+1))
print(f'acc(+-1cm): {acc3:.3f}')

--------------
new model
--------------
explained_variance_score: 0.804
mean_squared_errors: 1.448
mean_absolute_errors: 0.860
r2_score: 0.804
acc(+-1cm): 0.680


In [49]:
# save model
odir_f = f'mae-{mae:.3f}_mse-{mse:.3f}_RF-5inputs_{nfold}fold'
odir = f'result/depth_8cm/{odir_f}'
if not os.path.exists(odir):
    os.mkdir(odir)
#model.save_model(f'{odir}/model.model')
pickle.dump(gs, open(f'{odir}/gridSearch','wb'))
pickle.dump(rf, open(f'{odir}/model','wb'))

# 모델에 대한 정보 txt로 저장
pickle.dump(param_dict, open(f'{odir}/param_dict', 'wb'))
f = open(f'{odir}/result.txt', 'w')
f.write(f'regression model')
f.write(f'best params: {gs.best_params_}')
f.write(f'explained_variance_score: {explained_variance_score(y_test, y_pred):.3f}')
f.write(f'mean_squared_errors: {mean_squared_error(y_test, y_pred):.3f}')
f.write(f'mean_absolute_errors: {mean_absolute_error(y_test, y_pred):.3f}')
f.write(f'r2_score: {r2_score(y_test, y_pred):.3f}')
f.close()

[CV 6/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.795 total time=  16.7s
[CV 1/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50;, score=0.798 total time=   3.9s
[CV 5/10] END bootstrap=True, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.792 total time=   7.7s
[CV 2/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=50;, score=0.814 total time=   3.7s
[CV 3/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.805 total time=   7.5s
[CV 4/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200;, score=0.808 total time=  14.8s
[CV 2/10] END bootstrap=True, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=50;, score=0.815 total time=   3.7s
[CV 2/10] END bootstrap

# DNN model

In [28]:
from keras import losses, metrics
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization, Dropout, Activation, Input
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, concatenate, Add
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import itertools as it
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats, interp
import os, sys, pickle, shutil
import pandas as pd
import numpy as np
import random, datetime, time
from sklearn.model_selection import KFold


# tensorflow 사용 시 seed 고정
def seed_everything(seed: int = 98):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
SEED = 98
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

In [52]:
x_train.shape, x_test.shape

((24179, 4), (6044, 4))

In [53]:
from sklearn.preprocessing import StandardScaler

# normalization
sc = StandardScaler()
x_train = sc.fit_transform(pd.DataFrame(x_train))
x_test = sc.transform(pd.DataFrame(x_test))


# fill missing value
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

imp = IterativeImputer().fit(x_train)
x_train_imputed = imp.transform(x_train)
x_test_imputed = imp.transform(x_test)

In [54]:
# folder
nfold = 10  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 500
rootdir = f"result/depth_8cm/DNN_depth_both_4inputs"

if not os.path.exists(rootdir):
    os.mkdir(rootdir)

# 모델에 대한 정보 txt로 저장
f = open(f'{rootdir}/README.txt', 'w')
f.write(f'model: DNN 2 layers, regression')
f.write(f'input: age, sex, height, weight, cuffed 유무, tube size  output: depth')
f.close()
    

# test_settings
layer_settings, test_settings = [], []


# hyperparamters pool
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [16, 32, 64, 128, 256, 512]
BATCH_SIZE = [32, 64, 128, 256, 512]
lr_opts = [0.001, 0.002, 0.0005]

print('start making test settings...', end='', flush=True)
# test settings
dnodes, dropouts = [], []
for i in range(2):
    dnodes.append(0)
    dropouts.append(0)


for dnode1 in dense_opts:
    for dropout1 in dropout_opts:
        for dnode2 in dense_opts:
            for dropout2 in dropout_opts:
                for batch_size in BATCH_SIZE:
                    for learning_rate in lr_opts:
                        test_settings.append([dnode1, dropout1, dnode2, dropout2, batch_size, learning_rate])                                   

                        
print('done')
print(datetime.datetime.now())

start making test settings...done
2023-02-22 22:42:02.741033


In [ ]:
from keras import metrics

seed_everything(SEED)

# random search for hyperparameter
ntrial = ntest
train_errs, val_errs = [] ,[]
test_acc, test_roc, test_prc = [], [], []
#test_rmse, test_mae, test_auc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    
    # total conv layers of the model
    nlayer = random.choice([1,2]) 
    # test settings
    dnodes[0], dropouts[0], dnodes[1], dropouts[1], batch_size, learning_rate = random.choice(test_settings)
    

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = f'batch{batch_size},'
    for i in range(nlayer):
        odir_f += f'dnodes{dnodes[i]}_dropout{dropouts[i]},'
    odir_f += f'lr{learning_rate}'
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)


    # build a model
    inp = Input(shape=(x_train.shape[1],))
    out = inp

    
    for i in range(nlayer):      
        out = Dense(dnodes[i], activation='relu')(out)
        out = Dropout(dropouts[i])(out)
    
    out = Dense(1)(out)


    model = Model(inputs=[inp], outputs=[out])
    model.save_weights(f'{odir}/initial_weights.hdf5')
        

    # 4-fold cv
    kfold = KFold(nfold)
    acc1s, acc3s, maes, mses = [], [], [], []

    switch = 0
    for fold, (train_mask, test_mask) in enumerate(kfold.split(y_train)):
        X_train = x_train_imputed[train_mask]
        X_test = x_train_imputed[test_mask] 
        
        Y_train = y_train[train_mask]
        Y_test = y_train[test_mask]


        # model 학습
        try:
            weightcache = f"{odir}/weights_{fold}.hdf5"
            model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=[])
            hist = model.fit(X_train, Y_train, validation_split=0.2, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')])

            model.load_weights(weightcache)
            y_pred = model.predict(X_test).flatten()
            
            mae = mean_absolute_error(Y_test, y_pred)    
            mse = mean_squared_error(Y_test, y_pred)
            
            maes.append(mae)
            mses.append(mse)

            print(f' ###{fold} fold : val mae {mae:.3f}, mse {mse:.3f}###')
            tf.keras.backend.clear_session()
            model.load_weights(f'{odir}/initial_weights.hdf5')

        except Exception as e:
            print(e)
            switch = 1
            shutil.rmtree(odir)
            itrial -= 1
            break

    if switch:
        switch = 0
        continue
    

    print(f'mae{np.mean(mses):.3f}_mse{np.mean(acc3):.3f}')
    open(odir+"/model.json", "wt").write(model.to_json())

    os.rename(odir, rootdir+f'/mae{np.mean(maes):.3f}_mse{np.mean(mses):.3f}_{odir_f}')
    tf.keras.backend.clear_session()

random search 0/500


2023-02-22 22:42:06.450488: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 22:42:08.863436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30702 MB memory:  -> device: 0, name: Tesla V100-DGXS-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2023-02-22 22:42:08.864179: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 142 MB memory:  -> device: 1, name: Tesla V100-DGXS-32GB, pci bus id: 0000:08:00.0, compute capability: 7.0
2023-02-22 22:42:08.864824: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

Epoch 1/100
542/544 [============================>.] - ETA: 0s - loss: 7.8501
Epoch 00001: val_loss improved from inf to 1.62585, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch32,dnodes512_dropout0.4,dnodes512_dropout0.2,lr0.001/weights_0.hdf5
544/544 [==============================] - 2s 3ms/step - loss: 7.8298 - val_loss: 1.6259
Epoch 2/100
530/544 [============================>.] - ETA: 0s - loss: 2.2930
Epoch 00002: val_loss did not improve from 1.62585
544/544 [==============================] - 1s 3ms/step - loss: 2.2865 - val_loss: 1.7056
Epoch 3/100
540/544 [============================>.] - ETA: 0s - loss: 2.1789
Epoch 00003: val_loss improved from 1.62585 to 1.48550, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch32,dnodes512_dropout0.4,dnodes512_dropout0.2,lr0.001/weights_0.hdf5
544/544 [==============================] - 2s 3ms/step - loss: 2.1768 - val_loss: 1.4855
Epoch 4/100
536/544 [============================>.] - ETA: 0s - loss: 2.0243
Epoc

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


540/544 [============================>.] - ETA: 0s - loss: 7.8579
Epoch 00001: val_loss improved from inf to 1.52027, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch32,dnodes512_dropout0.4,dnodes512_dropout0.2,lr0.001/weights_1.hdf5
544/544 [==============================] - 2s 3ms/step - loss: 7.8199 - val_loss: 1.5203
Epoch 2/100
526/544 [============================>.] - ETA: 0s - loss: 2.2773
Epoch 00002: val_loss did not improve from 1.52027
544/544 [==============================] - 2s 3ms/step - loss: 2.2721 - val_loss: 1.6854
Epoch 3/100
531/544 [============================>.] - ETA: 0s - loss: 2.1739
Epoch 00003: val_loss did not improve from 1.52027
544/544 [==============================] - 2s 3ms/step - loss: 2.1663 - val_loss: 1.5729
 ###1 fold : val mae 0.883, mse 1.576###
Epoch 1/100


...

57/68 [========================>.....] - ETA: 0s - loss: 171.3131
Epoch 00001: val_loss improved from inf to 134.23267, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_3.hdf5
68/68 [==============================] - 1s 4ms/step - loss: 166.2404 - val_loss: 134.2327
Epoch 2/100
59/68 [=========================>....] - ETA: 0s - loss: 102.0549
Epoch 00002: val_loss improved from 134.23267 to 62.48118, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_3.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 97.2469 - val_loss: 62.4812
Epoch 3/100
57/68 [========================>.....] - ETA: 0s - loss: 44.0452
Epoch 00003: val_loss improved from 62.48118 to 27.85103, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_3.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 41.4719 - val_loss: 27.8510
Epoch 4/100
58/

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


58/68 [========================>.....] - ETA: 0s - loss: 170.8041
Epoch 00001: val_loss improved from inf to 134.07509, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_4.hdf5
68/68 [==============================] - 1s 4ms/step - loss: 166.1466 - val_loss: 134.0751
Epoch 2/100
58/68 [========================>.....] - ETA: 0s - loss: 102.1695
Epoch 00002: val_loss improved from 134.07509 to 62.41833, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_4.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 97.0085 - val_loss: 62.4183
Epoch 3/100
57/68 [========================>.....] - ETA: 0s - loss: 43.7339
Epoch 00003: val_loss improved from 62.41833 to 27.84838, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_4.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 41.3397 - val_loss: 27.8484
Epoch 4/100
57/

/home/painstudy/anaconda3/envs/keras/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


59/68 [=========================>....] - ETA: 0s - loss: 170.3318
Epoch 00001: val_loss improved from inf to 134.08670, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_5.hdf5
68/68 [==============================] - 1s 4ms/step - loss: 166.1812 - val_loss: 134.0867
Epoch 2/100
58/68 [========================>.....] - ETA: 0s - loss: 102.1031
Epoch 00002: val_loss improved from 134.08670 to 62.41707, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_5.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 96.9875 - val_loss: 62.4171
Epoch 3/100
57/68 [========================>.....] - ETA: 0s - loss: 43.7634
Epoch 00003: val_loss improved from 62.41707 to 27.85546, saving model to result/depth_8cm/DNN_depth_both_4inputs/batch256,dnodes512_dropout0,lr0.0005/weights_5.hdf5
68/68 [==============================] - 0s 3ms/step - loss: 41.3786 - val_loss: 27.8555
Epoch 4/100
59/

# Corelation coefficient

In [12]:
import scipy.stats as stats
# sex
stats.spearmanr(x_test[:,1], y_test)

SpearmanrResult(correlation=0.03861307987455406, pvalue=0.002678583454782267)

In [20]:
stats.spearmanr(x_test_imputed[:,2], y_test)

SpearmanrResult(correlation=0.8975062136826104, pvalue=0.0)

In [21]:
stats.spearmanr(x_test[:,4], y_test)

SpearmanrResult(correlation=0.1583183985229819, pvalue=3.198205039865417e-35)